In [ ]:
from qm.qua import *

In [ ]:
readout_pulse = 'a'
control_pulse = 'a'
readout_element = {}
control_element = {}
param_ar = []
frame_lut = []
frequency_lut = []
amp_lut = []
threshold_lut = []
x = []
cond_ar = []
def wait_for_value(): pass
calc_delay = 0

## Main text

In [ ]:
declare fixed x 
declare bool cond

strict_timing:
	measure(readout_pulse, readout_element, demod(x))  
	cond = x > 0
	wait_for_value(control_element, cond)
	play(control_pulse, control_element, condition = cond) 


In [3]:
declare fixed x 
declare bool cond

cond = False
strict_timing:
	while cond == False:
		measure(readout_pulse, readout_element, demod((x))  
		cond = x > 0
		wait(calc_delay, readout_element)
	align(control_element, readout_element)
	play(control_pulse, control_element)



SyntaxError: invalid syntax (2370937979.py, line 1)

In [ ]:
declare fixed x 
declare fixed param

strict_timing:
	measure(readout_pulse, readout_element, demod((x))  
	param = classical_calculation(x)
	wait_for_value(readout_element, param)
	update_param(param, control_element)
	play(response, control_element)  


## BM1.1 RT-D conditional

In [1]:
def benchmark_rtd_conditional_distributed(N_inout):
    
    fixed[N_inout] x
    bool[N_inout] cond_ar

    strict_timing:
        for i in range(N_inout):  # metaprogramming - unroll loop
            measure(readout_pulse, readout_element[i], demod(x[i]))
            cond_ar[i] = x[i] > 0
            wait_for_value(control_element[i], cond_ar[i])
            play(control_pulse, control_element[i], condition=cond_ar[i])
            


def benchmark_rtd_conditional_aggregated(N_inout):
    
    fixed[N_inout] x
    bool[N_inout] cond_ar
    bool cond

    strict_timing:
        for i in range(N_inout):  # metaprogramming - unroll loop
            measure(readout_pulse, readout_element[i], demod(x[i]))
            cond_ar[i] = x[i] > 0
        cond = and(cond_ar)  # metaprogramming - calculate `and` between all elements of cond_ar

        for i in range(N_inout):
            wait_for_value(control_element[i], cond)
            play(control_pulse, control_element[i], condition=cond)


def benchmark_rtd_conditional_aggregated_int(N_inout):
    
    fixed[N_inout] x
    bool cond

    strict_timing:
        for i in range(N_inout):  # metaprogramming - unroll loop
            measure(control_pulse, readout_element[i], demod(x[i]))

        cond = sum(x) > 0  # metaprogramming - calculate sum and apply threshold

        for i in range(N_inout):
            wait_for_value(control_element[i], cond)
            play(control_pulse, control_element[i], condition=cond)


IndentationError: expected an indented block (3034286722.py, line 3)

## BM1.2 RT-D flow control

In [ ]:
def benchmark_rtd_control_flow_distributed(N_inout):
    
    fixed[N_inout] x
    bool[N_inout] cond_ar

    for (i, 0, i < N_inout, i+1):
        cond_ar[i] = False

    strict_timing:
        for i in range(N_inout):  # metaprogramming - unroll loop
            while cond_ar[i] == False:
                wait_for_value(cond_ar[i], readout_element[i])
                measure(readout_pulse, readout_element[i], demod(x[i]))
                cond_ar[i] = x[i] > 0

            wait_for_value(cond_ar[i], control_element[i])
            play(control_pulse, control_element[i])
            


def benchmark_rtd_control_flow_aggregated(N_inout):
    
    declare fixed[N_inout] x
    declare bool[N_inout] cond_ar
    declare bool cond

    for (i, 0, i < N_inout, i+1):
        cond_ar[i] = False

    cond = False
    strict_timing:
        while cond == False:  
            for i in range(N_inout):  # metaprogramming - calculate `and` between all elements of cond_ar
                wait_for_value(cond, readout_element[i])
                measure(readout_pulse, readout_element[i], demod(x[i]))
                cond_ar[i] = x[i] > 0
            cond = and(cond_ar)

        wait_for_value(cond, control_element[i])
        for i in range(N_inout):
            play(control_pulse, control_element[i])


## BM1.3 RT-D parametric

In [ ]:
def benchmark_rtd_parametric_distributed_frame(N_inout):

    fixed[N_inout] x
    fixed[N_inout] param_ar

    strict_timing:
        for i in range(N_inout):
            measure(readout_pulse, readout_element[i], demod(x[i]))
            param_ar[i] = frame_lut[x > 0]
            wait_for_value(control_element[i], param_ar[i])
            frame_rotation_2pi(param_ar[i], control_element[i])
            play(control_pulse, control_element[i])


def benchmark_rtd_parametric_aggregated_frame(N_inout):

    fixed[N_inout] x
    bool[N_inout] cond_ar
    fixed param

    strict_timing:
        for i in range(N_inout):
            measure(readout_pulse, readout_element[i], demod(x[i]))
            cond_ar[i] = x[i] > 0
        param = bin2dec(cond_ar)

        param = param / 2**N_inout
        for i in range(N_inout):
            wait_for_value(control_element[i], param)
            frame_rotation_2pi(param, control_element[i])
            play(control_pulse, control_element[i])


def benchmark_rtd_parametric_distributed_frequency(N_inout):

    fixed[N_inout] x
    int[N_inout] param_ar

    strict_timing:
        for i in range(N_inout):
            measure(readout_pulse, readout_element[i], demod(x[i]))
            param_ar[i] = frame_lut[x > 0]
            wait_for_value(control_element[i], param_ar[i])
            update_frequency(param_ar[i], control_element[i])
            play(control_pulse, control_element[i])


def benchmark_rtd_parametric_aggregated_frequency(N_inout):

    fixed[N_inout] x
    bool[N_inout] cond_ar
    int param

    strict_timing:
        for i in range(N_inout):
            measure(readout_pulse, readout_element[i], demod(x[i]))
            cond_ar[i] = x[i] > 0
        param = bin2dec(cond_ar)

        param = 10 * param * param

        for i in range(N_inout):
            wait_for_value(control_element[i], param)
            update_frequency(param, control_element[i])
            play(control_pulse, control_element[i])


def benchmark_rtd_parametric_distributed_amp(N_inout):

    int[N_inout] x
    fixed[N_inout] param_ar

    strict_timing:
        for i in range(N_inout):
            measure(readout_pulse, readout_element[i], demod(x[i]))
            param_ar[i] = frame_lut[x > 0]
            wait_for_value(control_element[i], param_ar[i])
            play(control_pulse * amp(param_ar[i]), control_element[i])


def benchmark_rtd_parametric_aggregated_amp(N_inout):

    int[N_inout] x
    bool[N_inout] cond_ar
    fixed param

    strict_timing:
        for i in range(N_inout):
            measure(readout_pulse, readout_element[i], demod(x[i]))
            param = bin2dec(cond_ar)

        param = param / 2**N_inout

        for i in range(N_inout):
            wait_for_value(control_element[i], param)
            play(control_pulse * amp(param), control_element[i])


def benchmark_rtd_parametric_distributed_threshold(N_inout):

    int[N_inout] x
    int[N_inout] param_ar

    strict_timing:
        for i in range(N_inout):
            measure(readout_pulse, readout_element[i], demod(x[i]))
            param_ar[i] = frame_lut[x > 0]
            measure(readout_pulse, readout_element[i], demod(x[i]))
            result = x[i] > param_ar[i]
            wait_for_value(control_element[i], param_ar[i])
            play(control_pulse, control_element[i], condition=result)


## BM 2 Embedded near-real-time quantum-classical latency benchmark

In [ ]:
def parametric_benchmark(N_q, N_p, N_shots):
 
    declare bool[N_q] m
    declare fixed[N_p][2**N_q] T
    declare fixed[N_p] P
    declare int[2^N_q] H
 
    for (i, 0, i < 2**N_q, i+1): 
        H[i] = 0
 
    for (i, 0, i < N_shots, i+1):
        prep_qubits()
        run_calibration_circuit()
        m = measure_qubits()  # return binary string in decimal format
        H[m] ++
    P = T*H
 
    for (i, 0, i < N_p, i+1):
        update_parameter(param[i], P[i])
    # feedback latency is measured here
    prep_qubits()
    run_application_circuit()

from qm.QuantumMachinesManager import QuantumMachinesManager

